In [86]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\happy\AppData\Local\Temp\ipykernel_14164\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [87]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [88]:
## Read Data for the Fraudulent Email Kaggle Challenge
data_test = pd.read_csv(r"C:\Users\happy\Documents\ironhack\RecoveryPlan\Week9\day2\lab-natural-language-processing\data\kg_test.csv",encoding='latin-1')
data_train = pd.read_csv(r"C:\Users\happy\Documents\ironhack\RecoveryPlan\Week9\day2\lab-natural-language-processing\data\kg_train.csv",encoding='latin-1')



### Let's divide the training and test set into two partitions

In [89]:
# Reduce the training set to speed up development. 
# Modify for final system
data_train = data_train.head(1000)
print(data_train.shape)
data_train.fillna("",inplace=True)

(1000, 2)


In [90]:
# Reduce the training set to speed up development. 
# Modify for final system
data_test = data_test.head(1000)
print(data_test.shape)
data_test.fillna("",inplace=True)

(1000, 1)


In [91]:
print(data_test.columns)
print(data_train.columns)

Index(['text'], dtype='object')
Index(['text', 'label'], dtype='object')


In [92]:

vectorizer = TfidfVectorizer(max_features=5000)  # or CountVectorizer
X_train = vectorizer.fit_transform(data_train['text'])
X_test = vectorizer.transform(data_test['text'])

y_train = data_train['label']

In [93]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)

predicted_labels = model.predict(X_test)
data_test['predicted_label'] = predicted_labels

print(data_test.head())
data_test.to_csv("predicted_test_with_labels.csv", index=False)


                                                text  predicted_label
0  usiness is for the fact that the deceased man ...                1
1  They are happy to adjust to the afternoon. I a...                0
2  Lael Brainard was confirmed 78-19 this afterno...                0
3  H <hrod17@clintonemail.com>Friday March 26 201...                0
4  n;"> Dear Good Friend,<br><br><br>I am happy t...                1


## Data Preprocessing

In [94]:
import string
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [95]:
from bs4 import BeautifulSoup
import re

def clean_html(raw_html):
    # Step 1: Remove script and style tags (inline JS/CSS)
    soup = BeautifulSoup(raw_html, "html.parser")
    for script_or_style in soup(["script", "style"]):
        script_or_style.decompose()

    # Step 2: Get the text
    text = soup.get_text(separator=' ')

    # Step 3: Remove HTML comments (e.g. <!-- ... -->)
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)

    # Step 4: Remove leftover HTML tags (in case any remain)
    text = re.sub(r'<[^>]+>', '', text)

    return text

data_train['clean_text'] = data_train['text'].apply(clean_html)
data_test['clean_text'] = data_test['text'].apply(clean_html)

C:\Users\happy\AppData\Local\Temp\ipykernel_14164\3537460467.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(raw_html, "html.parser")
C:\Users\happy\AppData\Local\Temp\ipykernel_14164\3537460467.py:6: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(raw_html, "html.parser")
C:\Users\happy\AppData\Local\Temp\ipykernel_14164\3537460467.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(raw_html, "html.parser")


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [96]:
data_train

,text,label,clean_text
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL..."
1,Will do.,0,Will do.
2,Nora--Cheryl has emailed dozens of memos about...,0,Nora--Cheryl has emailed dozens of memos about...
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,Dear Sir=2FMadam=2C I know that this proposal ...
4,fyi,0,fyi
...,...,...,...
995,So what's the latest? It sounds contradictory ...,0,So what's the latest? It sounds contradictory ...
996,"TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...",1,"TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO..."
997,Barb I will call to explain. Are you back in t...,0,Barb I will call to explain. Are you back in t...
998,Yang on travelNot free tonite.May work tomorrow,0,Yang on travelNot free tonite.May work tomorrow


In [97]:
import re

def full_clean(text):
    # Remove special characters
    text = re.sub(r'[^A-Za-z0-9\s]', ' ', text)
    # Remove digits (numbers)
    text = re.sub(r'\d+', '', text)
    # Remove single characters (e.g., " a b c ")
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # Remove single characters from the start
    text = re.sub(r'^[a-zA-Z]\s+', '', text)

    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

  # Remove byte-string prefix b''
    text = re.sub(r"^b['\"]", '', text)

    # Convert to lowercase
    text = text.lower().strip()

    return text

data_train['clean_text'] = data_train['clean_text'].apply(full_clean)
data_test['clean_text'] = data_test['clean_text'].apply(full_clean)


## Now let's work on removing stopwords
Remove the stopwords.

In [98]:
def text_preprocessing_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

data_train['clean_text'] = data_train['clean_text'].apply(text_preprocessing_stopwords)
data_test['clean_text'] = data_test['clean_text'].apply(text_preprocessing_stopwords)


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [99]:
data_train

,text,label,clean_text
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,dear sir strictly private business proposal mi...
1,Will do.,0,
2,Nora--Cheryl has emailed dozens of memos about...,0,nora cheryl emailed dozens memos haiti weekend...
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,dear sir fmadam know proposal might surprise e...
4,fyi,0,fyi
...,...,...,...
995,So what's the latest? It sounds contradictory ...,0,latest sounds contradictory af decide shall ta...
996,"TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...",1,transfer million pounds youraccount name mr ej...
997,Barb I will call to explain. Are you back in t...,0,barb call explain back country h
998,Yang on travelNot free tonite.May work tomorrow,0,yang travelnot free tonite may work tomorrow


In [100]:
data_test

,text,predicted_label,clean_text
0,usiness is for the fact that the deceased man ...,1,usiness fact deceased man foreigner authorized...
1,They are happy to adjust to the afternoon. I a...,0,happy adjust afternoon going suggest pm start ...
2,Lael Brainard was confirmed 78-19 this afterno...,0,lael brainard confirmed afternoon miguel rodri...
3,H <hrod17@clintonemail.com>Friday March 26 201...,0,friday march sbwhoeop extended congrats
4,"n;""> Dear Good Friend,<br><br><br>I am happy t...",1,dear good friend happy inform succe ssin getti...
...,...,...,...
995,"Dear Friend,Do accept my sincere apologies if ...",1,dear friend accept sincere apologies mail meet...
996,FROM THE DESK OF:MR.WANG QINHANG SENG BANK LTD...,1,desk mr wang qinhang seng bank ltd des voeux r...
997,He can speak at 7pm. He said that it does not ...,0,speak pm said secure ops connect residence
998,Mary Landrieu asked me to attend a dinner abou...,0,mary landrieu asked attend dinner foster care ...


In [101]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download required resources (only once)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')  # WordNet synonyms

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()
def tokenize_and_lemmatize(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]
    return lemmatized_tokens
data_train['lemmatized'] = data_train['clean_text'].apply(tokenize_and_lemmatize)
data_test['lemmatized'] = data_test['clean_text'].apply(tokenize_and_lemmatize)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\happy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\happy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\happy\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [102]:
from collections import Counter

# Separate messages by label
ham_messages_train = data_train[data_train['label'] == 0]['lemmatized']
spam_messages_train = data_train[data_train['label'] == 1]['lemmatized']
ham_messages_test = data_test[data_test['predicted_label'] == 0]['lemmatized']
spam_messages_test = data_test[data_test['predicted_label'] == 1]['lemmatized']

# Combine all words into single lists
ham_words_train = [word for message in ham_messages_train for word in message]
spam_words_train = [word for message in spam_messages_train for word in message]
ham_words_test = [word for message in ham_messages_test for word in message]
spam_words_test = [word for message in spam_messages_test for word in message]

# Count word frequencies
ham_freq_train = Counter(ham_words_train)
spam_freq_train = Counter(spam_words_train)
ham_freq_test = Counter(ham_words_test)
spam_freq_test = Counter(spam_words_test)

# Get top 10 most common words
top_10_ham_train = ham_freq_train.most_common(10)
top_10_spam_train = spam_freq_train.most_common(10)
top_10_ham_test = ham_freq_test.most_common(10)
top_10_spam_test = spam_freq_test.most_common(10)

# Display
print("Top 10 Ham Words (Train):")
print("Top 10 Ham Words:")
for word, count in top_10_ham_train:
    print(f"{word}: {count}")

print("Top 10 Spam Words (Train):")
print("\nTop 10 Spam Words:")
for word, count in top_10_spam_train:
    print(f"{word}: {count}")

print("Top 10 Ham Words (Test):")
print("Top 10 Ham Words:")
for word, count in top_10_ham_test:
    print(f"{word}: {count}")

print("Top 10 Spam Words (Test):")
print("\nTop 10 Spam Words:")
for word, count in top_10_spam_test:
    print(f"{word}: {count}")


Top 10 Ham Words (Train):
Top 10 Ham Words:
state: 136
pm: 127
would: 107
president: 99
time: 95
call: 94
mr: 91
obama: 84
percent: 81
secretary: 78
Top 10 Spam Words (Train):

Top 10 Spam Words:
money: 987
account: 899
bank: 801
fund: 782
u: 733
transaction: 553
business: 514
country: 513
mr: 488
million: 455
Top 10 Ham Words (Test):
Top 10 Ham Words:
pm: 164
state: 128
secretary: 110
obama: 108
clinton: 95
call: 94
president: 93
time: 87
u: 85
said: 83
Top 10 Spam Words (Test):

Top 10 Spam Words:
e: 1479
money: 976
account: 861
bank: 816
fund: 743
u: 659
mr: 507
country: 497
transaction: 482
business: 461


## Extra features

In [103]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
# money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
# suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

# data_train['money_mark'] = data_train['lemmatized'].fillna('').str.contains(money_simbol_list)*1
# data_train['suspicious_words'] = data_train['lemmatized'].fillna('').str.contains(suspicious_words)*1
# data_train['text_len'] = data_train['lemmatized'].fillna('').apply(lambda x: len(x))

# data_test['money_mark'] = data_test['lemmatized'].fillna('').str.contains(money_simbol_list)*1
# data_test['suspicious_words'] = data_test['lemmatized'].fillna('').str.contains(suspicious_words)*1
# data_test['text_len'] = data_test['lemmatized'].fillna('').apply(lambda x: len(x))

# data_train['money_mark'] = data_train['lemmatized'].str.contains(money_simbol_list)*1
# data_train['suspicious_words'] = data_train['lemmatized'].str.contains(suspicious_words)*1
# data_train['text_len'] = data_train['lemmatized'].apply(lambda x: len(x)) 

# data_test['money_mark'] = data_test['lemmatized'].str.contains(money_simbol_list)*1
# data_test['suspicious_words'] = data_test['lemmatized'].str.contains(suspicious_words)*1
# data_test['text_len'] = data_test['lemmatized'].apply(lambda x: len(x)) 
# Define your keyword lists
money_keywords = ["euro", "dollar", "pound", "€", "$"]
suspicious_keywords = ["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"]

# Functions to check presence in the token list
def contains_money(tokens):
    if not isinstance(tokens, list):
        return 0
    return int(any(word in tokens for word in money_keywords))

def contains_suspicious(tokens):
    if not isinstance(tokens, list):
        return 0
    return int(any(word in tokens for word in suspicious_keywords))

# Apply to train and test
data_train['money_mark'] = data_train['lemmatized'].apply(contains_money)
data_train['suspicious_words'] = data_train['lemmatized'].apply(contains_suspicious)
data_train['text_len'] = data_train['lemmatized'].apply(lambda x: len(x) if isinstance(x, list) else 0)

data_test['money_mark'] = data_test['lemmatized'].apply(contains_money)
data_test['suspicious_words'] = data_test['lemmatized'].apply(contains_suspicious)
data_test['text_len'] = data_test['lemmatized'].apply(lambda x: len(x) if isinstance(x, list) else 0)


data_test.head()

,text,predicted_label,clean_text,lemmatized,money_mark,suspicious_words,text_len
0,usiness is for the fact that the deceased man ...,1,usiness fact deceased man foreigner authorized...,"[usiness, fact, deceased, man, foreigner, auth...",0,1,108
1,They are happy to adjust to the afternoon. I a...,0,happy adjust afternoon going suggest pm start ...,"[happy, adjust, afternoon, going, suggest, pm,...",0,0,16
2,Lael Brainard was confirmed 78-19 this afterno...,0,lael brainard confirmed afternoon miguel rodri...,"[lael, brainard, confirmed, afternoon, miguel,...",0,0,17
3,H <hrod17@clintonemail.com>Friday March 26 201...,0,friday march sbwhoeop extended congrats,"[friday, march, sbwhoeop, extended, congrats]",0,0,5
4,"n;""> Dear Good Friend,<br><br><br>I am happy t...",1,dear good friend happy inform succe ssin getti...,"[dear, good, friend, happy, inform, succe, ssi...",0,1,119


## How would work the Bag of Words with Count Vectorizer concept?

In [104]:
# Your code

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [105]:
# Example: apply to both train and test
data_train['joined_text'] = data_train['lemmatized'].apply(lambda tokens: ' '.join(tokens))
data_test['joined_text'] = data_test['lemmatized'].apply(lambda tokens: ' '.join(tokens))

from sklearn.feature_extraction.text import CountVectorizer

# Initialize vectorizer
vectorizer = CountVectorizer()

# Fit on training data and transform
X_train_vectorized = vectorizer.fit_transform(data_train['joined_text'])

# Transform test data using the same vectorizer
X_test_vectorized = vectorizer.transform(data_test['joined_text'])

print("Training shape:", X_train_vectorized.shape)
print("Test shape:", X_test_vectorized.shape)


Training shape: (1000, 21698)
Test shape: (1000, 21698)


## And the Train a Classifier?

In [106]:
data_train

,text,label,clean_text,lemmatized,money_mark,suspicious_words,text_len,joined_text
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,dear sir strictly private business proposal mi...,"[dear, sir, strictly, private, business, propo...",1,1,203,dear sir strictly private business proposal mi...
1,Will do.,0,,[],0,0,0,
2,Nora--Cheryl has emailed dozens of memos about...,0,nora cheryl emailed dozens memos haiti weekend...,"[nora, cheryl, emailed, dozen, memo, haiti, we...",0,0,17,nora cheryl emailed dozen memo haiti weekend p...
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,dear sir fmadam know proposal might surprise e...,"[dear, sir, fmadam, know, proposal, might, sur...",1,1,186,dear sir fmadam know proposal might surprise e...
4,fyi,0,fyi,[fyi],0,0,1,fyi
...,...,...,...,...,...,...,...,...
995,So what's the latest? It sounds contradictory ...,0,latest sounds contradictory af decide shall ta...,"[latest, sound, contradictory, af, decide, sha...",0,0,8,latest sound contradictory af decide shall tak...
996,"TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...",1,transfer million pounds youraccount name mr ej...,"[transfer, million, pound, youraccount, name, ...",1,1,235,transfer million pound youraccount name mr eji...
997,Barb I will call to explain. Are you back in t...,0,barb call explain back country h,"[barb, call, explain, back, country, h]",0,0,6,barb call explain back country h
998,Yang on travelNot free tonite.May work tomorrow,0,yang travelnot free tonite may work tomorrow,"[yang, travelnot, free, tonite, may, work, tom...",0,1,7,yang travelnot free tonite may work tomorrow


In [107]:
data_test

,text,predicted_label,clean_text,lemmatized,money_mark,suspicious_words,text_len,joined_text
0,usiness is for the fact that the deceased man ...,1,usiness fact deceased man foreigner authorized...,"[usiness, fact, deceased, man, foreigner, auth...",0,1,108,usiness fact deceased man foreigner authorized...
1,They are happy to adjust to the afternoon. I a...,0,happy adjust afternoon going suggest pm start ...,"[happy, adjust, afternoon, going, suggest, pm,...",0,0,16,happy adjust afternoon going suggest pm start ...
2,Lael Brainard was confirmed 78-19 this afterno...,0,lael brainard confirmed afternoon miguel rodri...,"[lael, brainard, confirmed, afternoon, miguel,...",0,0,17,lael brainard confirmed afternoon miguel rodri...
3,H <hrod17@clintonemail.com>Friday March 26 201...,0,friday march sbwhoeop extended congrats,"[friday, march, sbwhoeop, extended, congrats]",0,0,5,friday march sbwhoeop extended congrats
4,"n;""> Dear Good Friend,<br><br><br>I am happy t...",1,dear good friend happy inform succe ssin getti...,"[dear, good, friend, happy, inform, succe, ssi...",0,1,119,dear good friend happy inform succe ssin getti...
...,...,...,...,...,...,...,...,...
995,"Dear Friend,Do accept my sincere apologies if ...",1,dear friend accept sincere apologies mail meet...,"[dear, friend, accept, sincere, apology, mail,...",1,1,152,dear friend accept sincere apology mail meet p...
996,FROM THE DESK OF:MR.WANG QINHANG SENG BANK LTD...,1,desk mr wang qinhang seng bank ltd des voeux r...,"[desk, mr, wang, qinhang, seng, bank, ltd, de,...",1,1,265,desk mr wang qinhang seng bank ltd de voeux rd...
997,He can speak at 7pm. He said that it does not ...,0,speak pm said secure ops connect residence,"[speak, pm, said, secure, ops, connect, reside...",0,0,7,speak pm said secure ops connect residence
998,Mary Landrieu asked me to attend a dinner abou...,0,mary landrieu asked attend dinner foster care ...,"[mary, landrieu, asked, attend, dinner, foster...",0,0,29,mary landrieu asked attend dinner foster care ...


In [108]:
y_train = data_train['label']  # or whatever your label column is named
y_test = data_test['predicted_label']  # or whatever your label column is named
from scipy.sparse import hstack

# Select other features, for example:
other_features_train = data_train[['money_mark', 'suspicious_words', 'text_len']].values
other_features_test = data_test[['money_mark', 'suspicious_words', 'text_len']].values

# Combine sparse text features and dense other features
import numpy as np
from scipy.sparse import csr_matrix

# Convert dense numeric features to sparse format for hstack
other_features_train_sparse = csr_matrix(other_features_train)
other_features_test_sparse = csr_matrix(other_features_test)

# Combine text and other features
X_train_vec = hstack([X_train_vectorized, other_features_train_sparse])
X_test_vec = hstack([X_test_vectorized, other_features_test_sparse])

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

#Initialize Multinomial Naive Bayes classifier with default parameters
clf = MultinomialNB()

#Train the classifier on training data
clf.fit(X_train_vec, y_train)

#Predict on the test set
y_pred = clf.predict(X_test_vec)

#Evaluate performance
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.87      0.91       539
           1       0.86      0.95      0.91       461

    accuracy                           0.91      1000
   macro avg       0.91      0.91      0.91      1000
weighted avg       0.91      0.91      0.91      1000



In [109]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 1. Split data_train into train and val
train_df, val_df = train_test_split(data_train, test_size=0.2, random_state=42, stratify=data_train['label'])

# 2. Prepare train features and labels
train_df['joined_text'] = train_df['lemmatized'].apply(lambda tokens: ' '.join(tokens))
X_train_text = train_df['joined_text']
y_train = train_df['label']
other_features_train = train_df[['money_mark', 'suspicious_words', 'text_len']].values

# 3. Prepare val features and labels
val_df['joined_text'] = val_df['lemmatized'].apply(lambda tokens: ' '.join(tokens))
X_val_text = val_df['joined_text']
y_val = val_df['label']
other_features_val = val_df[['money_mark', 'suspicious_words', 'text_len']].values

# 4. Fit vectorizer on train text only
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train_text)

# 5. Vectorize val text
X_val_vectorized = vectorizer.transform(X_val_text)

# 6. Combine text and numeric features
from scipy.sparse import csr_matrix, hstack
X_train_vec = hstack([X_train_vectorized, csr_matrix(other_features_train)])
X_val_vec = hstack([X_val_vectorized, csr_matrix(other_features_val)])

# 7. Train classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_vec, y_train)

# 8. Predict on validation set
y_pred = clf.predict(X_val_vec)

# 9. Evaluate performance
print(classification_report(y_val, y_pred))


              precision    recall  f1-score   support

           0       0.99      0.89      0.94       112
           1       0.88      0.99      0.93        88

    accuracy                           0.94       200
   macro avg       0.93      0.94      0.93       200
weighted avg       0.94      0.94      0.94       200



In [110]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

# Since you have other features (money_mark, suspicious_words, text_len), you can’t directly
# pass them to a simple pipeline that vectorizes only text, so cross-validation is a bit more complex.

# If you want to do cross-validation only on text features:
X_text = data_train['lemmatized'].apply(lambda tokens: ' '.join(tokens))
y = data_train['label']

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

scores = cross_val_score(pipeline, X_text, y, cv=5, scoring='f1_macro')
print("Mean F1 macro score:", scores.mean())


Mean F1 macro score: 0.9406123015366156


In [111]:
from sklearn.metrics import classification_report, f1_score

def search_best_features_with_joined_text(data, text_col='joined_text', label_col='label'):
    # Split train/validation
    train_df, val_df = train_test_split(
        data, test_size=0.2, random_state=42, stratify=data[label_col])
    
    X_train_text = train_df[text_col]
    X_val_text = val_df[text_col]
    
    y_train = train_df[label_col]
    y_val = val_df[label_col]
    
    # Numeric features
    numeric_cols = ['money_mark', 'suspicious_words', 'text_len']
    num_features_train = train_df[numeric_cols].values
    num_features_val = val_df[numeric_cols].values
    
    vectorizer_options = [
        ('CountVectorizer unigram', CountVectorizer(ngram_range=(1,1))),
        ('CountVectorizer bigram', CountVectorizer(ngram_range=(1,2))),
        ('CountVectorizer unigram stopwords', CountVectorizer(ngram_range=(1,1), stop_words='english')),
        ('TfidfVectorizer unigram', TfidfVectorizer(ngram_range=(1,1))),
        ('TfidfVectorizer bigram', TfidfVectorizer(ngram_range=(1,2))),
        ('TfidfVectorizer unigram stopwords', TfidfVectorizer(ngram_range=(1,1), stop_words='english')),
    ]
    
    results = []
    
    for name, vectorizer in vectorizer_options:
        # Vectorize text
        X_train_vec = vectorizer.fit_transform(X_train_text)
        X_val_vec = vectorizer.transform(X_val_text)
        
        # Combine text vector and numeric features
        X_train_combined = hstack([X_train_vec, csr_matrix(num_features_train)])
        X_val_combined = hstack([X_val_vec, csr_matrix(num_features_val)])
        
        # Train classifier
        clf = MultinomialNB()
        clf.fit(X_train_combined, y_train)
        
        # Predict on validation
        y_pred = clf.predict(X_val_combined)
        
        # Evaluation
        f1 = f1_score(y_val, y_pred)
        print(f"=== {name} ===")
        print(classification_report(y_val, y_pred))
        print(f"F1 Score: {f1:.4f}")
        print('-'*50)
        
        results.append((name, f1))
    
    # Summary sorted by F1 score
    print("\nSummary of F1 Scores:")
    for name, f1 in sorted(results, key=lambda x: x[1], reverse=True):
        print(f"{name}: {f1:.4f}")

# Example usage:
search_best_features_with_joined_text(data_train, text_col='joined_text', label_col='label')


=== CountVectorizer unigram ===
              precision    recall  f1-score   support

           0       0.99      0.89      0.94       112
           1       0.88      0.99      0.93        88

    accuracy                           0.94       200
   macro avg       0.93      0.94      0.93       200
weighted avg       0.94      0.94      0.94       200

F1 Score: 0.9305
--------------------------------------------------
=== CountVectorizer bigram ===
              precision    recall  f1-score   support

           0       0.99      0.64      0.78       112
           1       0.69      0.99      0.81        88

    accuracy                           0.80       200
   macro avg       0.84      0.82      0.79       200
weighted avg       0.85      0.80      0.79       200

F1 Score: 0.8093
--------------------------------------------------
=== CountVectorizer unigram stopwords ===
              precision    recall  f1-score   support

           0       0.99      0.89      0.94       

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [112]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 1. Split data_train into train and val
train_df, val_df = train_test_split(data_train, test_size=0.2, random_state=42, stratify=data_train['label'])

# 2. Prepare train features and labels
train_df['joined_text'] = train_df['lemmatized'].apply(lambda tokens: ' '.join(tokens))
X_train_text = train_df['joined_text']
y_train = train_df['label']
other_features_train = train_df[['money_mark', 'suspicious_words', 'text_len']].values

# 3. Prepare val features and labels
val_df['joined_text'] = val_df['lemmatized'].apply(lambda tokens: ' '.join(tokens))
X_val_text = val_df['joined_text']
y_val = val_df['label']
other_features_val = val_df[['money_mark', 'suspicious_words', 'text_len']].values

# 4. Fit vectorizer on train text only
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train_text)

# 5. Vectorize val text
X_val_vectorized = vectorizer.transform(X_val_text)

# 6. Combine text and numeric features
from scipy.sparse import csr_matrix, hstack
X_train_vec = hstack([X_train_vectorized, csr_matrix(other_features_train)])
X_val_vec = hstack([X_val_vectorized, csr_matrix(other_features_val)])

# 7. Train classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_vec, y_train)

MultinomialNB()

In [114]:
new_email = "Congratulations! You won a free prize. Please send your bank account details."
print(predict_email(new_email))

spam


In [ ]:
import re
import string
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download once globally if you haven't yet:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

money_keywords = ["euro", "dollar", "pound", "€", "$"]
suspicious_keywords = [
    "free", "cheap", "sex", "money", "account", "bank",
    "fund", "transfer", "transaction", "win", "deposit", "password"
]

def clean_html(raw_html):
    soup = BeautifulSoup(raw_html, "html.parser")
    for script_or_style in soup(["script", "style"]):
        script_or_style.decompose()
    text = soup.get_text(separator=' ')
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    text = re.sub(r'<[^>]+>', '', text)
    return text

def full_clean(text):
    text = re.sub(r'[^A-Za-z0-9\s]', ' ', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'^[a-zA-Z]\s+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r"^b['\"]", '', text)
    return text.lower().strip()

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

def tokenize_and_lemmatize(text):
    tokens = word_tokenize(text)
    return [lemmatizer.lemmatize(token.lower()) for token in tokens]

def contains_money(tokens):
    if not isinstance(tokens, list):
        return 0
    return int(any(word in tokens for word in money_keywords))

def contains_suspicious(tokens):
    if not isinstance(tokens, list):
        return 0
    return int(any(word in tokens for word in suspicious_keywords))

# Define preprocessing function for a new email text
def preprocess_email(email_text):
    """
    Preprocess the raw email text by:
    - cleaning HTML and text,
    - removing special chars, digits, stopwords,
    - tokenizing,
    - lemmatizing tokens,
    and finally returning a list of tokens.
    """
    
    # Clean HTML and remove scripts/styles/comments
    text = clean_html(email_text)
    
    # Remove special chars, digits, single chars, normalize spaces, lowercase
    text = full_clean(text)
    
    # Remove stopwords
    text = remove_stopwords(text)
    
    # Tokenize and lemmatize
    tokens = tokenize_and_lemmatize(text)
    
    return tokens

# Define function to extract numeric features from raw text (money_mark, suspicious_words, text_len)
import pandas as pd

money_symbol_list = ["euro","dollar","pound","€","$"]
suspicious_words_list = ["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"]

def extract_numeric_features(text):
    text_lower = text.lower()
    money_mark = int(any(sym in text_lower for sym in money_symbol_list))
    suspicious_mark = int(any(word in text_lower for word in suspicious_words_list))
    text_len = len(text.split())
    return [money_mark, suspicious_mark, text_len]

# Define function to predict new email
def predict_email(email_text):
    # Preprocess and join tokens
    tokens = preprocess_email(email_text)
    joined_text = ' '.join(tokens)
    
    # Vectorize text using fitted vectorizer
    text_vectorized = vectorizer.transform([joined_text])
    
    # Extract numeric features
    numeric_features = extract_numeric_features(email_text)
    numeric_features_sparse = csr_matrix([numeric_features])
    
    # Combine features
    features = hstack([text_vectorized, numeric_features_sparse])
    
    # Predict label (0=ham,1=spam)
    pred = clf.predict(features)[0]
    
    return "spam" if pred == 1 else "ham"

# # Example usage:
# new_email = "Congratulations! You won a free prize. Please send your bank account details."
# print(predict_email(new_email))


spam


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\happy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\happy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\happy\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\happy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
